In [ ]:
using Jutul, JutulDarcy, Plots
nx = ny = 10
nz = 2
day = 3600*24
bar = 1e5
g = CartesianMesh((nx, ny, nz), (2000.0, 1500.0, 50.0))
Darcy = 9.869232667160130e-13
K = repeat([0.1*Darcy], 1, number_of_cells(g))
res = discretized_domain_tpfv_flow(tpfv_geometry(g), porosity = 0.1, permeability = K)
# Vertical well in (1, 1, *), producer in (nx, ny, 1)
P = setup_vertical_well(g, K, 1, 1, name = :Producer)
I = setup_well(g, K, [(nx, ny, 1)], name = :Injector)
# Set up a two-phase immiscible system
phases = (AqueousPhase(), VaporPhase())
rhoWS = 1000.0; rhoGS = 700.0
rhoS = [rhoWS, rhoGS]
sys = ImmiscibleSystem(phases, reference_densities = rhoS)
model, parameters = setup_reservoir_model(res, sys, wells = [I, P])
# Replace the density function with our custom version for wells and reservoir
c = [1e-6/bar, 1e-5/bar]
ρ = ConstantCompressibilityDensities(p_ref = 1*bar, density_ref = rhoS, compressibility = c)
replace_variables!(model, PhaseMassDensities = ρ)
# Set up report time-steps, five years with 30 days each
dt = repeat([30.0]*day, 12*5)
# Inject two full pore-volumes (at reference conditions) of gas
rate_target = TotalRateTarget(2.0*sum(pore_volume(model))/sum(dt))
bhp_target = BottomHolePressureTarget(50*bar)
controls = Dict(:Injector => InjectorControl(rate_target, [0.0, 1.0], density = rhoGS),
                :Producer => ProducerControl(bhp_target))
# Wrap forces and initialize the state
forces = setup_reservoir_forces(model, control = controls)
state0 = setup_reservoir_state(model, Pressure = 150*bar, Saturations = [1.0, 0.0])
sim, config = setup_reservoir_simulator(model, state0, parameters)
states, reports = simulate!(sim, dt, forces = forces, config = config)
# Get output and plot results
wo = full_well_outputs(model, states, forces)
T = report_times(reports)./day
qgs = abs.(wo[:Producer][Symbol("Surface gas rate")])
qws = abs.(wo[:Producer][Symbol("Surface water rate")])
plt = plot(T, qgs, xlabel = "Days", ylabel="m³/s", label = "Gas rate")
plot!(T, qws, label = "Water rate")
display(plt)